In [518]:
import random

In [519]:
with open('msg.txt', 'r', encoding='utf8') as f:
  msg = f.read()

In [520]:
print(msg)

Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?


In [521]:
with open('cover.txt', 'r', encoding='utf-8') as f:
    cover = f.read()

In [522]:
print(cover)

xin chao cac ban



In [523]:
def to_bits(s: str) -> str:
    result: str = ""
    for c in s:
        bits = bin(ord(c))[2:]
        bits = bits.zfill(8)
        result += bits
    return result

In [524]:
def key_gen(msg: str, key_file: str) -> str:
    msg_len = len(to_bits(msg))
    key = ''.join([str(random.randint(0, 1)) for i in range(msg_len)])
    with open(key_file, 'w') as f:
        f.write(key)
    return key


def xor_str(str1: str, str2: str) -> str:
    assert len(str1) == len(str2)
    result = ""
    for i in range(len(str1)):
        assert str1[i] == '0' or str1[i] == '1'
        assert str2[i] == '0' or str2[i] == '1'
        xor_ed = int(str1[i]) ^ int(str2[i])
        result += str(xor_ed)
    return result


def encrypt(message: str, key: str) -> str:
    assert len(message) == len(key)
    return xor_str(message, key)


def decrypt(encrypted: str, key: str) -> str:
    assert len(encrypted) == len(key)
    return xor_str(encrypted, key)

In [525]:
invisible = {
    "\u200B",
    "\u200E",
    "\u202C",
    "\u202A",
    "\u202B",
    "\u202C",
    "\u202D",
}

bits_to_zwc = {
    "0001": "\u200B\u202A",
    "0010": "\u202A\u200B",
    "0011": "\u202C\u202A",
    "0100": "\u202A\u202C",
    "0101": "\u202C\u202D",
    "0110": "\u200B\u202C",
    "0111": "\u202C\u200B",
    "1000": "\u202A\u202B",
    "1001": "\u202B\u202A",
    "1010": "\u200E\u202D",
    "1011": "\u202D\u200E",
    "1100": "\u200E\u202C",
    "1101": "\u202C\u200E",
    "1110": "\u202D\u202C",
    "1111": "\u202D\u200B",
    "0000": "\u200B\u202D"
}

zwc_to_bits = {bits_to_zwc[x]: x for x in bits_to_zwc.keys()}

In [526]:
def invert(s: str) -> str:
    result = ""
    for c in s:
        if c == "0":
            result += "1"
        else:
            result += "0"
    return result

In [527]:
def from_bits(bits: str) -> str:
    chars: str = ""
    for b in range(len(bits) // 8):
        byte = bits[b * 8:(b + 1) * 8]
        chars += chr(int(''.join([bit for bit in byte]), 2))
    return chars

In [528]:
def separate(bits: str) -> list[str]:
    result: list[str] = []
    for b in range(len(bits) // 4):
        result.append(bits[b * 4: (b + 1) * 4])
    return result

In [529]:
def xor_pair(first: str, second: str) -> str:
    assert len(first) == len(second)
    return bin(int(first, 2) ^ int(second, 2))[2:].zfill(4)

In [530]:
def xor_separated_bits(separated_bits: list[str]) -> list[str]:
    result: list[str] = []
    for b in range(0, len(separated_bits), 2):
        result.append(xor_pair(separated_bits[b], separated_bits[b + 1]))
    return result

In [531]:
def merged_xor(separated_bits: list[str], xor_ed_bits: list[str]) -> list[str]:
    result: list[str] = []
    for b in range(0, len(separated_bits), 2):
        result.append(separated_bits[b] + xor_ed_bits[b // 2])
    return result

In [532]:
def map_bits_to_zwc(bits: str, mapped: dict[str, str]) -> str:
    assert len(bits) == 4
    return mapped[bits]

In [533]:
def encode_to_zwc(merged_xor_ed: list[str]) -> str:
    result = ""
    for bits in merged_xor_ed:
        result += map_bits_to_zwc(bits[:4], bits_to_zwc)
        result += map_bits_to_zwc(bits[4:], bits_to_zwc)
    return result

In [534]:
def extract_bits_from_stego(stego_text: str, mapped: dict[str, str]) -> str:
    hidden_char = ""
    for c in stego_text:
        if c in invisible:
            hidden_char += c
    bits = ""
    for c_index in range(0, len(hidden_char), 2):
        bits += mapped[hidden_char[c_index] + hidden_char[c_index + 1]]
    return bits


In [535]:
def embed(msg_file: str, cover_file: str, stego_file: str = None, key=None):
    assert msg_file.endswith('.txt') and cover_file.endswith('.txt')
    if stego_file:
        assert stego_file.endswith('txt')
    with open(msg_file, 'r', encoding='utf-8') as f:
        msg = f.read()

    with open(cover_file, 'r', encoding='utf-8') as f:
        cover = f.read()

    msg_bits = to_bits(msg)

    if key:
      msg_bits = encrypt(msg_bits, key)

    inverted = invert(msg_bits)
    separated = separate(inverted)
    xor_ed = xor_separated_bits(separated)
    merged_xor_ed = merged_xor(separated, xor_ed)
    encoded = encode_to_zwc(merged_xor_ed)
    stego_text = encoded + cover

    out_file = stego_file
    if not out_file:
        file = cover_file.split('.')[0]
        out_file = file + '_embedded.txt'
    with open(out_file, 'w', encoding='utf-8') as f:
        f.write(stego_text)

In [536]:
def extract(embedded_file: str, output_msg_file: str = None, key=None):
    with open(embedded_file, 'r', encoding='utf-8') as f:
        stego_text = f.read()
    bits = extract_bits_from_stego(stego_text, zwc_to_bits)
    bits_separated = separate(bits)
    xor_ed = xor_separated_bits(bits_separated)
    merged_xor_ed = merged_xor(bits_separated, xor_ed)

    msg_bits = invert("".join(merged_xor_ed))
    if key:
      msg_bits = decrypt(msg_bits, key)
    
    decoded = from_bits(msg_bits)

    out_file = output_msg_file
    if not out_file:
        file = embedded_file.split('.')[0]
        out_file = file + '_extracted.txt'
    with open(out_file, 'w', encoding='utf-8') as f:
        f.write(decoded)


In [537]:
key = key_gen(msg, 'key')
print(key)

0011001000110101011111000001010110011010101001101010100001101011110100110101101000001001011001101000101011100000000000011110111010101111000001111111110001000011010010101110101101100100111101010000000000001010110111001110000101000100001101111110100000111101110110010100001101001000111111100110100101110101010100011101000100001110011100011110001101000010010001011010101010011100111101101010011110110101111100000111110000100110100010011110000101011001100101110100111101000010011101100101011001101001001011100010011010011100111011001011111110110000110110011000011101011110001001110000000001101111000011100000101100001010101000000101101011011011001010110110010100011110000111010000001011111001101011111001001111110001011101100101000111000000001000101001100011110100010110010001101101111010000001000001101111110000111110110101111001000001011111110111001111011110001111101010111001110010011000011000001111000100110001011010110001100111111000001111100100001100000001110011101110110011000100111111001000011101

In [538]:
embed('msg.txt', 'cover.txt', 'embedded.txt', key)

In [539]:
with open('embedded.txt', 'r', encoding='utf8') as f:
  embeded = f.read()

In [540]:
print(embeded)

‫‪‬​‎‭‬‭‭‬‫‪‎‬​‬​‪​‪‪​‭​‬​​‭‭‬‎‭‪‬‬‎‬‎‎‭‪‫‬‎‭‬‬​​‪‬‎‬​‭‎‫‪‭‬‬​‬​‪​​‬‫‪‪‫‬​‬​‫‪‬‭‎‬‎‬‬​‬‎‭​​‭​‬‫‪‬‎‪​‫‪‬‪‪‬‎‭‬​‬​‬‎‭​‭‎​‭‬‪‪‬‎‭​‪‬‭​‭‎‬‪‬‬‎‭​‪​‬‪‭​‬​‭‬‬‭‬‎‬​‬‭‭‬‪‫‎‭‎‭‪‬‬​‭‬‎‬‬‪‎‬‪‬‬‪‫‪​‪​‭‪​‭‎‪​‫‪‪​​‪‬​‎‬‎‭‫‪‎‭‬‭​‪‪‫‬​‬‭‬‎‭‬​‪‫‪‎‬‪‫‬‎​‪‭​‪​‎‬​‭‭​‪‬‭​‭‬‭‎‬‪​‭​‭‬​‬​‬‪​‬‬‪‭​‪‬‬‪​‪‬​‬‎‪‫‭‎‎‭‪‫‪​‭​‪​‬‎‎‬‫‪‫‪‫‪‬‪‬‪​‭‎‬​‬‬‭‬‪‭‎‎‭‭​‪‫‫‪‫‪‫‪‭‬‫‪​‪‪​‪‬‬‪‭​​‭‬‎‬​‎‬‭‬‬‪‎‬‬‪‬‭‪‬‎‭‬‪​‭‭‬‬​‫‪‎‬‬​‎‬‪‬‎‭‭​‪‫‬​‪‫‬‪‬​‎‬​‬‬‪‎‬​‭‫‪‬​‭​‬‎‭‬‬​‪‬‪‫‭‬‭​‬‪‬‪‭​‪​‭​‪‬​‭‭‬‬‭‬​‬‭‭‬‬‪‬‎‭‎‭​‬‪‎‬​‬‬‭‫‪‭‎‪‫​‪‭‎‎‭‪​‭‎‎‬​‭​‬‪​‫‪‭‎‬​‭​‪​‭​​‭‭‎‭‬‫‪​‪‭​​‭‬‪‎‬‬‎‬‎‭‎‭‎‭‬‪‬‭‎‭​‪​‫‪​‪‪​‭‎‬​​‭​‭‬‭‭‎‭‬‪‬‎‭​‪‭‎‭​‭‎‬‭​‭‭‎‪‫‪​‬‪‬‪‬​‪‬​‬​‭‭‬‪​‬​‬‭​‭‪‬‭​‪​‪​‭​‫‪​‪‭‎‭‬‬‪‬‪‪‫​‪‭‬​‬‭​‫‪‭‎​‭‎‬​‪​‭‎‭‬​‬‭‫‪‪‬‎‬​‪‎‬‭​‫‪​‬‭‬‭‬‬‎​‬‬​​‭‎‭‬‪‬‭‎‬‬‎‪​‎‬‫‪‭​‬‎‪​‪‫‪‫‪‬‎‭‬‪​‪‫‪‎‬‭​‬‎‎‭‬‎‭‎​‬‎‬‭‬‭‎‪‬‭‬‬‪​‪‪‫‭‎‭‬​‪​‭‭​‫‪‬‎‭​​‬‭​‎‭‬‪‪‬‬‭‎‬‎‬‭​‪‬‪​‭​‎‭‭‬‬‪‎‬‭‎‪‫​‭‭‎​‬‪‬‭‬‬​‎‬‎‬‫‪‪‫‭‎‪​‭‬‬‎‬‎‫‪‭‬​‪‬‎‭‎‪​‭​​‪‎‬‬‎‭​​‭‬​‫‪‭​‭‬‪‬‭​​‪‎‭‭​‬‎‪‬‪​‫‪‭​​‬​‪​‬‬‎‬‪‪‬​‪‬​​‪‭‎‭‬‬‭‭‎‬‪‭‬‫‪‭‎‪​‬‎‫‪‫‪‎‬‎‬‎‬‭‬​‭‎‬‬‭‬‭‎‭‭​‪‬‫‪‪​‭​‪​‬‪​‬​‪‎‭‎‬‭‎‬​‫‪‭‬‎‭‪‬​‬‫‪‭‎​‬‬‭‎‭‎‭‬‎‪‫‫‪

In [541]:
extract('embedded.txt', "extracted.txt", key)

In [542]:
with open('extracted.txt', 'r', encoding='utf8') as f:
  extracted = f.read()

In [543]:
print(extracted)

Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?


In [544]:
print(msg)

Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?


In [545]:
assert extracted == msg